# MOS

Compute a Model Output Statistic (MOS) post-processing model for our GDPS data.

MOS is recognized as a gold-standard baseline for post-processing weather forecasts.
It's a simple linear model from predictors to the output quantity. 
In our case we will output the Gaussian distribution of temperature (so expected mean and STD).
Our predictions will be scored using CRPS which is deemed the best way to evaluate the quality of a distribution over observations.

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
import dask.distributed
import dask_jobqueue
import math
import numpy as np
import os
import pathlib
import torch
import torch.nn as nn

import seaborn as sns
import xarray as xr

## Boot dask cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

## Read dataset

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
INPUT_DIR = DATA_DIR / '2021-03-17-ppdataset/'

In [ ]:
df = dd.read_parquet(DATA_DIR / '2021-03-17-ppdataset/*.parquet')

In [ ]:
df['step_hour'] = df['step'] / 3600
df['valid'] = df['date'] + dd.to_timedelta(df['step'], unit='S')
df['forecast_hour'] = df['date'].dt.hour

We will be working with only one lead time at first, say 48hrs.

In [ ]:
lead_48 = df[df['step_hour'] == 48]

In [ ]:
df

In [ ]:
lead_48

In [ ]:
lead_48.count().compute()

## Build big array with 3 axis: station time feature

The strategy is to iterate on the stations a build an xarray dataset for every station.
Then we merge the xarray datasets into a big one.

In [ ]:
stations = lead_48['station'].value_counts().compute()

In [ ]:
stations_to_keep = stations[stations > 1400].index

In [ ]:
lead_48_lots_obs = lead_48[lead_48['station'].isin(stations_to_keep)]

In [ ]:
len(stations_to_keep)

In [ ]:
lead_48_lots_obs_compute = lead_48_lots_obs.compute()

In [ ]:
feature_columns = [c for c in lead_48.columns if c.startswith('gdps')]
feature_columns.append("obs_2t")

In [ ]:
datasets = []

for station in stations_to_keep:
    station_obs = lead_48_lots_obs_compute[lead_48_lots_obs_compute['station'] == station]
    station_obs.sort_values('date')
        
    features = station_obs[feature_columns].to_numpy()
    
    data_arrays = {}
    for feature in feature_columns:
        data_array = xr.DataArray(np.expand_dims(station_obs[feature].to_numpy(), axis=0), dims=['station', 'date'])
        data_arrays[feature] = data_array
        
    dataset = xr.Dataset(
        data_arrays,
        coords={
            'station': xr.DataArray([station], dims=['station']),
            'date': xr.DataArray(station_obs['date'], dims=['date'])
        })
    
    datasets.append(dataset)

In [ ]:
merged = xr.concat(datasets, dim='station')

In [ ]:
merged.sel(date=slice("2019-01-01", "2019-12-31")).isnull().sum(dim="station")

In [ ]:
(merged.isnull().sum(dim='station') == 0).sum()

## Learn on a train set

In [ ]:
train_slice = slice("2019-01-01", "2019-12-31")
train_dataset = merged.drop('obs_2t').sel(date=train_slice)
train_y = merged.obs_2t.sel(date=train_slice)

In [ ]:
val_dataset = merged.drop('obs_2t').sel(date=slice("2020-01-01", "2020-12-31"))

In [ ]:
train_array = train_dataset.to_array().transpose('station', 'date', 'variable')

In [ ]:
train_array.shape

In [ ]:
train_y.shape

In [ ]:
class MOS(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        
        self.mu = nn.Linear(in_features, 1, bias=True)
        self.sigma = nn.Linear(in_features, 1, bias=True)
        
    def forward(self, x):
        return self.mu(x), self.sigma(x)

In [ ]:
model = MOS(20)

In [ ]:
def crps_loss(mu_pred, sigma_pred, y_true):
    pi = torch.Tensor([math.pi])
    
    """CRPS for a Normal distribution."""
    loc = (mu_pred - y_true) / sigma_pred
    phi = 1.0 / torch.sqrt(2.0 * pi) * torch.exp(-torch.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + torch.erf(loc / torch.sqrt(torch.Tensor([2.0]))))
    
    crps = torch.sqrt(torch.square(sigma_pred)) * (loc * (2. * Phi - 1.) + 2 * phi - 1. / torch.sqrt(pi))
    
    return torch.mean(crps)

In [ ]:
train_tensor = torch.from_numpy(train_array.data).float()

In [ ]:
train_tensor[train_tensor.isnan()] = 0.0

In [ ]:
train_y = torch.from_numpy(train_y.data).unsqueeze(dim=-1).float()

In [ ]:
mu_hat, sigma_hat = model(train_tensor)

In [ ]:
crps_loss(mu_hat, sigma_hat, train_y)

In [ ]:
mu_hat.shape

In [ ]:
sigma_hat.shape

In [ ]:
train_y.shape

In [ ]:
train_tensor.dtype

In [ ]:
mu_hat.isnan().sum()

In [ ]:
sigma_hat.isnan().sum()

In [ ]:
train_y.isnan().sum()

In [ ]:
train_y.shape